# Autproductie

## Overzicht
We kijken naar de dataset `cars.csv`, deze bevat informatie over de verkoop van 2e hands Volvo’s. We hebben hier een aantal vragen over, beantwoord deze vragen door een beetje code te schrijven en de antwoorden op te schrijven in een Jupyter Notebook.

### Vraag 1:
Wat voor soort data zijn `year`, `fuel_type` en `seller_rating`? Motiveer je antwoord.

<span style="color:CadetBlue">  

- atribuut year bevat categorische ordinale data  

motivatie : het attibuut is niet numerisch vermits jaren niet opgeteld kunnen worden en is daarom eerder categorisch, ook omdat jaren niet steeds even groot zijn. Opgelet: met jaren kunnen tijdsafhankelijke berekeningen worden uitgevoerd, in dat kader kàn het numerisch zijn. Niet in het kader van deze vraag en oefening, naar mijn mening. Het is ordinaal omdat jaren een logische - in dit geval chronologische - volgorde hebben.

- atribuut fueltype bevat categorische niet ordinale data  

motivatie : na onderzoek is vast te stellen dat de gegevens in het atribuut fueltype van het nominale type zijn vermits er geen ranschikking of volgorde mogelijk. Er is wel wat overlap in betekenis van de gebruikte waardes, alsook vreemde data zoals 'automatic'

- atribuut seller_rating bevast categorische ordinale data

motivatie : ratings zijn niet numeriek in dit geval vermits ze niet opgeteld of numerisch benaderd kunnen worden. Dat heeft geen zin. Ze zijn hier categorisch (tussen waarden 1.0 en 5.0) en ordinaal omdat een rating van 5.0 een 'betere' ofte hogere waardering geeft 

</span>

### Vraag 2:
Ik vraag me af hoe een gewoonlijke (mbt prijs) $2^e$ hands Volvo eruit ziet. Kan je deze zoeken en tonen?

<span style="color:CadetBlue">  

Ik heb geen idee wat hier precies gevraagd wordt.
Wordt hier gevraagd hoe een Volvo er uit ziet ?
Wordt hier gevraagd hoe een Volvo er uit ziet met betrekking tot (mbt) de prijs ? Wat is dat voor een onozele vraag ?

</span>

### Vraag 3:
We kunnen de verdeling van de prijs van $2^e$ hands Volvo's voorstellen met een histogram:
![alt text](image.png)
 - Waar zou je verwachten dat het gemiddelde +/- ligt voor deze verdeling?
 - Waar zou je verwachten dat de mediaan +/- ligt?

Teken dit histogram met `.hist` en gebruik `.axvline` om de mediaan en het gemiddelde toe te voegen aan de plot. Komt dit overeen met je verwachting?

 - Kan je de emprisiche cummulatieve distributie functie tekenen voor de verdeling die overeenkomt met dit histogram (zie ook `scipy.stats.ecdf`).

### Vraag 4:
Ik wil nu de verschillende fuel types vergelijken hoeveel deze kosten. Hiervoor maak ik volgende afbeelding met meerdere histogrammen hoe kunnen we deze beter maken? Welke alternatieven zijn er nog?
![alt text](image-1.png)

### Vraag 5:
Gegeven volgende boxplots van de prijs van elektrische & benzine wagens, zijn volgende uitspraken waar/onwaar/niet te besluiten & leg uit:

![alt text](image-2.png)

 - De spreiding op de prijs van elektrische wagens is groter dan die van benzinewagens.

 - Een wagen die meer dan 50 000 euro kost heeft een grotere kans om elektrisch te zijn dan benzine.

 - Een elektrische wagen heeft een grotere kans om meer dan 50 000 euro te kosten dan een benzine wagen.

### Vraag 6:
Maak een visualizatie om het verband tussen mileage & price te tonen?

### Vraag 7:
Volgens deze data is de gemiddelde prijs van $2^e$ hands elektrische Volvo’s 48 000 euro; als we veronderstellen dat onze dataset representatief is voor alle 2de hands volvo’s, hoe kan je de accuraatheid van deze schatting nagaan?